In [ ]:
import sys

import pandas as pd
import numpy as np
import random
import seaborn as sns
import time
import warnings
import re
from collections import Counter
from IPython.display import FileLink, FileLinks
from datetime import datetime

import matplotlib.pyplot as plt
import matplotlib as mpl
import scipy.stats as stats
from matplotlib import colors
from matplotlib.cbook import boxplot_stats
from mpl_toolkits.mplot3d import Axes3D
from tqdm import tqdm
from tqdm import tqdm_notebook
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.patches import Patch

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import SelectKBest, chi2

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.base import BaseEstimator, TransformerMixin


from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, classification_report, precision_score, recall_score, accuracy_score

warnings.filterwarnings("ignore")
random.seed(168)
tqdm.pandas()
sns.set(rc={"axes.facecolor":"#FFF9ED","figure.facecolor":"#FFF9ED"})
pallet = ["#ea5545", "#f46a9b", "#ef9b20", "#edbf33", "#ede15b", "#bdcf32", "#87bc45", "#27aeef", "#b33dc6"]
pallet_prot = ['#00429d', '#2854a6', '#3e67ae', '#507bb7', '#618fbf', '#73a2c6', '#85b7ce', '#9acbd5', '#b1dfdb', '#cdf1e0', '#ffe5cc', '#ffcab9', '#ffaea5', '#fd9291', '#f4777f', '#e75d6f', '#d84360', '#c52a52', '#ae1045', '#93003a']
base_path = '/home/lamle3105/Project_Churn/'
# Refactor CustomerID
def normalize_customer_number(cn):
    if len(str(cn)) == 8 :
        return str(cn)
    else :
        return (8 - len(str(cn)))*"0" + str(cn)

In [ ]:
# Load DL Feature KH
df = pd.read_csv("dataset/Customer_Churn_Attributes_20231016.csv", sep="\t", index_col=0)
df["CustomerNumber"] = df["CustomerNumber"].apply(lambda x : normalize_customer_number(x))
print(df.shape)
df.head(10)

In [ ]:
# Count occurrences of each churn category
churn_counts = df['isChurn'].value_counts()

labels = churn_counts.index
sizes = churn_counts.values

# Define colors
colors = ['#ff9999','#66b3ff']  # Adjust as needed

plt.figure(figsize=(7, 7))
wedges, texts, autotexts = plt.pie(sizes, labels=labels, autopct='%1.1f%%', startangle=90, colors=colors)

legend_labels = [f"{label}: {count}" for label, count in zip(labels, sizes)]
plt.legend(wedges, legend_labels, title="Churn Count", loc="best")

# Add a title
plt.title('Churn Distribution')

# Display the chart
plt.show()


In [ ]:
gender_counts = df['Gender'].value_counts()

labels = gender_counts.index
sizes = gender_counts.values

colors = ['#ff9999','#66b3ff', '#99ff99', '#ffcc99']

plt.figure(figsize=(7, 7))
wedges, texts, autotexts = plt.pie(sizes, labels=labels, autopct='%1.1f%%', startangle=90, colors=colors)
legend_labels = [f"{label}: {count}" for label, count in zip(labels, sizes)]
plt.legend(wedges, legend_labels, title="Gender Count", loc="best")

plt.title('Gender Distribution')

plt.show()

In [ ]:
# Count total and unique values
total_rows = df.shape[0]
unique_customers = df["CustomerNumber"].nunique()
total_duplicates = df.duplicated().sum()
duplicates_excluding_customer = df.drop("CustomerNumber", axis=1).duplicated().sum()

# Print summary
print(f"Total Rows: {total_rows}")
print(f"Total Unique Customers: {unique_customers}")
print(f"Total Duplicate Rows (Including Customer Number): {total_duplicates}")
print(f"Total Duplicate Rows (Excluding Customer Number): {duplicates_excluding_customer}")

In [ ]:
# Calculate null counts and percentages
null_counts = df.isnull().sum()
null_percentage = (null_counts / len(df)) * 100

# Filter columns that have null values
null_data = pd.DataFrame({"Null Count": null_counts, "Null Percentage": null_percentage})
null_data = null_data[null_data["Null Count"] > 0]  # Keep only columns with nulls

# Print the result
print(null_data.sort_values(by="Null Percentage", ascending=False))

In [ ]:
df_drop_na = df.dropna(subset='MembershipType')
columns_fill_value = {"CountNotiRead": 0,
                      "CountNotiSended": 0,
                      "AvgRatingPoint": -1,
                      "MinRatingPoint": -1,
                      "Age": int(np.mean(df_drop_na["Age"])),
                      "Gender": "UNK",
                      "FirstJoinProvince": "UNK",
                      "FirstJoinRegion": "UNK",
                      "TotalBurnedEarn": 0,
                      "AvgGcoinEarn": 0,
                      "AvgBurnedEarn": 0,
                      "TGS_Avartar": 0,
                      "TGS_MomoRegister": 0}
df_fill_na = df_drop_na.fillna(value=columns_fill_value)
mean_values = df_fill_na.groupby("MembershipType")["TotalGcoinEarn"].transform("mean")
df_fill_na["TotalGcoinEarn"] = df_fill_na["TotalGcoinEarn"].fillna(mean_values)
df_fill_na.shape

In [ ]:
null_counts = df_fill_na.isnull().sum()
null_percentage = (null_counts / len(df_fill_na)) * 100

# Filter columns that have null values
null_data = pd.DataFrame({"Null Count": null_counts, "Null Percentage": null_percentage})
null_data = null_data[null_data["Null Count"] > 0]  # Keep only columns with null values

# Sort and print the result
print(null_data.sort_values(by="Null Percentage", ascending=False))

In [ ]:
df_fill_na.info()

In [ ]:
df_fill_na["Gender"].value_counts().to_frame().T

In [ ]:
df_fill_na["Age"].value_counts().to_frame().T

In [ ]:
df_fill_na["MembershipType"].value_counts().to_frame().T

In [ ]:
df_fill_na["FirstJoinProvince"].value_counts().to_frame().T

In [ ]:
df_fill_na["FirstJoinRegion"].value_counts().to_frame().T

In [ ]:
df_fill_na["BrandMostVisited"].value_counts().to_frame().T

In [ ]:
df_fill_na["BrandLatestVisited"].value_counts().to_frame().T

In [ ]:
df_fill_na["TotalGcoinEarn"].value_counts().to_frame().T

In [ ]:
df_fill_na["CustomerNumber"] = df_fill_na["CustomerNumber"].astype(int)
df_fill_na["Age"] = df_fill_na["Age"].astype(int)
df_fill_na["TGS_Avartar"] = df_fill_na["TGS_Avartar"].astype(bool)
df_fill_na["TGS_MomoRegister"] = df_fill_na["TGS_MomoRegister"].astype(bool)

In [ ]:
df_reform = df_fill_na.copy()

# Check loyalty : Silver, Gold, Diamond only 
df_reform = df_reform[df_reform["MembershipType"].isin(["GPeople Silver", "GPeople Gold", "GPeople Diamond"])]

# Reformat Brand Text -> Categories  
def normalize_brand(s):
    s = re.sub(r'[^\w\s]', '', s)
    s = re.sub(" ", '', s)
    s = re.sub("\t", '', s)
    s = s.strip()
    return s.lower()
df_reform["BrandMostVisited"] = df_reform["BrandMostVisited"].apply(lambda x : normalize_brand(x))
df_reform["BrandLatestVisited"] = df_reform["BrandLatestVisited"].apply(lambda x : normalize_brand(x))